### Porting to Google Colab
The following cell enables this notebook to run from Google Colab as well as from your local machine IDE.<br>
You can change `root_directory` and/or `this_notebook_google_path` to point to the directory in your Google account, which contains this notebook, together with the `imgs` sub-directory and the rest of the files.<br>

In [1]:
import sys
import os
try:
    from google.colab import drive as google_drive # type: ignore
except:
    # no Google Colab --> fall back to local machine
    google_drive = None

if google_drive is not None:
    google_drive_directory = os.path.join('/','content','gdrive')
    google_drive.mount(google_drive_directory)
    all_projects_path = os.path.join(google_drive_directory, 'Othercomputers','My Laptop', 'projects')
else:
    all_projects_path = os.path.join('d:\\', 'projects')

project_path = os.path.join(all_projects_path,'RUNI','Thesis')
assert os.path.exists(project_path), f'Project path {project_path} not found!'
# enable import python files from this notebook's path
sys.path.append(project_path)
# enable reading images and data files from this notebook's path
os.chdir(project_path)

datasets_path = os.path.join(project_path, 'datasets')
assert os.path.exists(datasets_path), f'Datasets path {datasets_path} not found!'

output_path = os.path.join(project_path, 'output')
os.makedirs(output_path, exist_ok=True)
assert os.path.exists(output_path), f'Output path {output_path} not found!'

print(f'Current working directory: {os.getcwd()}')
print(f'Datasets path: {datasets_path}')
print(f'Output path: {output_path}')

Current working directory: d:\projects\RUNI\Thesis
Datasets path: d:\projects\RUNI\Thesis\datasets
Output path: d:\projects\RUNI\Thesis\output


In [2]:
from python.hpc import HybridArray

Numba version: 0.60.0
numba.njit is available.
CUDA is available and will be used for GPU operations.
Printing CUDA active device attributes:
    Name:                               NVIDIA GeForce GTX 1650
    Free Memory:                        3367680 [KB]
    Total Memory:                       4193984 [KB]
    Compute capability:                 7.5
    Clock rate:                         1560.00 [MHz]
    Memory clock rate:                  4001.00 [MHz]
    Memory bus width:                   128 bits
    Memory band width (theoretical)     128.03 [GByte/Sec]
    Number of multiprocessors:          16
    Minimal grid size:                  128
    Maximum grid size:                  (2147483647, 65535, 65535)
    Maximum block dimensions:           (1024, 1024, 64)
    Maximum threads per block:          1024
    Warp size:                          32
    Maximum shared memory per block:    49152 [bytes]
    Maximum registers per block:        65536
    Total constant memory:   

In [3]:
import numpy as np
from python.metrics.metrics import analyze_auc_multi_tuples_n1_mu

Compiling Berk Jones for CUDA...
Compiling Berk Jones for CUDA... Done!


In [4]:
num_monte = 20
N = 100
n1s = (np.arange(0.01,0.11,0.01)*N).astype(np.uint32)
mus = np.arange(0.1,1.1,0.1,dtype=np.float64)
n1_mu_tuples = list(zip(n1s,mus))


In [5]:

def simulation3(num_monte: int, N: int, transform_method: str, discovery_method: str,\
                n1: np.uint32, mu: float = 1.0) -> None:
    print(f'Running on {transform_method=} {discovery_method=} {num_monte=} {N=} {n1=} {mu=}')
    for gpu,njit in [(False,False), (False,True), (True,False)]:
        auc =analyze_auc_multi_tuples_n1_mu(\
            N=N, num_monte=num_monte, transform_method=transform_method,\
            discovery_methods=discovery_method,\
            n1_mu_tuples=[(n1,mu)], use_gpu=gpu, use_njit=njit)
        print(f'{gpu=} {njit=} --> auc.mean()={auc.mean():.2f}')




In [6]:
simulation3(num_monte=num_monte, N=N, transform_method='higher_criticism',\
            discovery_method='argmin', n1=n1s[0], mu=mus[0])

Running on transform_method='higher_criticism' discovery_method='argmin' num_monte=20 N=100 n1=1 mu=0.1


Processing: 100%|██████████| 2/2 [00:00<00:00, 16.93step/s, Current Step=1] 


gpu=False njit=False --> auc.mean()=0.68


Processing: 100%|██████████| 2/2 [00:20<00:00, 10.15s/step, Current Step=1]


gpu=False njit=True --> auc.mean()=0.68


Processing:   0%|          | 0/2 [00:00<?, ?step/s, Current Step=0]d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
Processing:  50%|█████     | 1/2 [00:00<00:00,  1.20step/s, Current Step=1]d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 5 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\di

gpu=True njit=False --> auc.mean()=0.68


In [7]:
simulation3(num_monte=num_monte, N=N, transform_method='higher_criticism_unstable',\
            discovery_method='argmin', n1=n1s[0], mu=mus[0])

Running on transform_method='higher_criticism_unstable' discovery_method='argmin' num_monte=20 N=100 n1=1 mu=0.1


Processing: 100%|██████████| 2/2 [00:00<00:00, 19.84step/s, Current Step=1] 


gpu=False njit=False --> auc.mean()=0.61


Processing: 100%|██████████| 2/2 [00:01<00:00,  1.40step/s, Current Step=1]


gpu=False njit=True --> auc.mean()=0.61


Processing:   0%|          | 0/2 [00:00<?, ?step/s, Current Step=0]d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
Processing: 100%|██████████| 2/2 [00:00<00:00, 12.08step/s, Current Step=1]

gpu=True njit=False --> auc.mean()=0.61


In [8]:
simulation3(num_monte=num_monte, N=N, transform_method='berk_jones',\
            discovery_method='argmin', n1=n1s[0], mu=mus[0])

Running on transform_method='berk_jones' discovery_method='argmin' num_monte=20 N=100 n1=1 mu=0.1


Processing: 100%|██████████| 2/2 [00:00<00:00,  7.11step/s, Current Step=1]


gpu=False njit=False --> auc.mean()=0.68


Processing: 100%|██████████| 2/2 [00:00<00:00,  2.27step/s, Current Step=1]


gpu=False njit=True --> auc.mean()=0.68


Processing:   0%|          | 0/2 [00:00<?, ?step/s, Current Step=0]d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
Processing: 100%|██████████| 2/2 [00:00<00:00, 13.25step/s, Current Step=1]

gpu=True njit=False --> auc.mean()=0.68


In [9]:

def simulation3_multi(num_monte: int, N: int, transform_method: str,\
                    discovery_method: str,\
                        n1_mu_tuples: list) -> None:
    print(f'Running on {transform_method=} {discovery_method=} {num_monte=} {N=} {n1s=} {mus=}')
    for gpu,njit in [(False,False), (False,True), (True,False)]:
        auc = analyze_auc_multi_tuples_n1_mu(num_monte=num_monte, N=N, use_gpu=gpu, use_njit=njit,\
                       transform_method=transform_method,\
                       discovery_methods=discovery_method,\
                       n1_mu_tuples=n1_mu_tuples)
        print(f'{gpu=} {njit=} --> auc.mean()={auc.mean():.2f}')




In [10]:
simulation3_multi(num_monte=num_monte, N=N, transform_method='higher_criticism',\
            discovery_method='argmin',\
            n1_mu_tuples=n1_mu_tuples)

Running on transform_method='higher_criticism' discovery_method='argmin' num_monte=20 N=100 n1s=array([ 1,  2,  3,  4,  5,  6,  6,  8,  9, 10], dtype=uint32) mus=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])


Processing: 100%|██████████| 11/11 [00:01<00:00, 10.80step/s, Current Step=10]


gpu=False njit=False --> auc.mean()=0.59


Processing: 100%|██████████| 11/11 [00:00<00:00, 242.43step/s, Current Step=10]


gpu=False njit=True --> auc.mean()=0.59


Processing:  18%|█▊        | 2/11 [00:00<00:00, 126.19step/s, Current Step=2]d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
Processing:  36%|███▋      | 4/11 [00:00<00:00, 145.70step/s, Current Step=4]d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 3 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
Processing:  73%|███████▎  | 8/11 [00:00<00:00, 158.56step/s, Current Step=8]d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
Processing: 100%|██████████| 11/11 [00:00<00:00, 158.03step/s, Current Step=10]

gpu=True njit=False --> auc.mean()=0.59
